In [1]:
%cd ..

/home/nikita/edu/competitions/admet


In [2]:
from collections import deque
from tqdm import tqdm

import numpy as np
import pandas as pd

import torch
from torch import nn
import torch.nn.functional as F

from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GINConv, GATv2Conv, GCNConv, Sequential
from torch_geometric.nn.aggr import AttentionalAggregation, MeanAggregation

from rdkit import Chem
from rdkit.Chem import AllChem

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

In [3]:
def smiles_to_graph(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    atom_features = []
    for atom in mol.GetAtoms():
        atom_features.append(atom.GetAtomicNum())

    edges = []
    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        edges.append((i, j))
        edges.append((j, i))

    # Convert to PyTorch tensors
    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
    x = torch.tensor(atom_features, dtype=torch.float).view(-1, 1)

    return Data(x=x, edge_index=edge_index)

In [4]:
df_train = pd.read_csv("data/train_admet.csv", index_col=0)
df_test = pd.read_csv("data/test_data.csv", index_col=0)
sample = pd.read_csv("data/sample.csv")

In [5]:
df_trains = []
df_vals = []
df_tests = []

properties = df_train.property.unique()
for prop in properties:
    segment = df_train[df_train["property"] == prop]
    train, val = train_test_split(
        segment, test_size=0.2, random_state=75, stratify=segment.Y
    )
    df_trains.append(train)
    df_vals.append(val)
    df_tests.append(df_test[df_test["property"] == prop])

sampler = RandomOverSampler(random_state=0)

for i in range(len(df_trains)):
    df_trains[i] = sampler.fit_resample(df_trains[i], df_trains[i].Y)[0]
    df_vals[i] = sampler.fit_resample(df_vals[i], df_vals[i].Y)[0]

In [6]:
train_datasets = []
val_datasets = []

for i in range(len(df_trains)):
    train_data = []
    for j, row in df_trains[i].iterrows():
        features = smiles_to_graph(row["Drug"])
        target = row["Y"]

        features.y = target
        train_data.append(features)
    train_datasets.append(train_data)

    val_data = []
    for j, row in df_vals[i].iterrows():
        features = smiles_to_graph(row["Drug"])
        target = row["Y"]

        features.y = target
        val_data.append(features)
    val_datasets.append(val_data)

[08:03:01] WARNING: not removing hydrogen atom without neighbors
[08:03:01] WARNING: not removing hydrogen atom without neighbors
[08:03:01] WARNING: not removing hydrogen atom without neighbors
[08:03:01] WARNING: not removing hydrogen atom without neighbors
[08:03:01] WARNING: not removing hydrogen atom without neighbors
[08:03:01] WARNING: not removing hydrogen atom without neighbors
[08:03:01] WARNING: not removing hydrogen atom without neighbors
[08:03:01] WARNING: not removing hydrogen atom without neighbors
[08:03:01] WARNING: not removing hydrogen atom without neighbors
[08:03:01] WARNING: not removing hydrogen atom without neighbors
[08:03:01] WARNING: not removing hydrogen atom without neighbors
[08:03:01] WARNING: not removing hydrogen atom without neighbors
[08:03:01] WARNING: not removing hydrogen atom without neighbors
[08:03:01] WARNING: not removing hydrogen atom without neighbors
[08:03:01] WARNING: not removing hydrogen atom without neighbors
[08:03:01] WARNING: not r

In [7]:
train_dataloaders = []
val_dataloaders = []

for i in range(len(train_datasets)):
    train_dataloaders.append(DataLoader(train_datasets[i], batch_size=32, shuffle=True))
    val_dataloaders.append(DataLoader(val_datasets[i], batch_size=32, shuffle=False))

/home/nikita/edu/competitions/admet/.conda/lib/python3.11/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [17]:
class MoleculeCrusher(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=100, embedding_dim=128)
        self.gnn = Sequential(
            "x, edge_index, batch",
            [
                (GCNConv(128, 256), "x, edge_index -> x"),
                nn.SiLU(),
                (GCNConv(256, 256), "x, edge_index -> x"),
                nn.SiLU(),
                (GCNConv(256, 512), "x, edge_index -> x"),
                nn.SiLU(),
                (GCNConv(512, 512), "x, edge_index -> x"),
                nn.SiLU(),
                (GCNConv(512, 1024), "x, edge_index -> x"),
                nn.SiLU(),
                (GCNConv(1024, 1024), "x, edge_index -> x"),
                nn.SiLU(),
                (
                    MeanAggregation(),
                    "x, batch -> x",
                ),
                nn.SiLU(),
                (nn.Linear(1024, 1), "x -> x"),
            ],
        )

        self.optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)

    @property
    def device(self):
        return next(self.parameters()).device

    def compute_loss(self, pred, target):
        return F.binary_cross_entropy_with_logits(pred, target)
    
    def compute_auc(self, pred, target):
        return roc_auc_score(target, pred)

    def forward(self, data: Data):
        x = data.x.view(-1).to(torch.long)
        idx = data.edge_index
        batch = data.batch

        x = self.embedding(x)
        x = self.gnn(x, idx, batch)

        return x.view(-1)

    @torch.no_grad()
    def predict(self, data: Data):
        return torch.sigmoid(self.forward(data))

    def train(self, epochs, train_dataloader, val_dataloader):
        for epoch in range(epochs):
            train_loss_buffer = deque(maxlen=4)
            train_loop = tqdm(train_dataloader, desc=f"Train epoch {epoch}")
            for data in train_loop:
                data = data.to(self.device)
                target = data.y.to(torch.float32)
                pred = self.forward(data)
                loss = self.compute_loss(pred, target)
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                train_loss_buffer.append(loss.item())
                train_loop.set_postfix(loss=np.mean(train_loss_buffer))

            val_loss_buffer = list()
            val_loop = tqdm(val_dataloader, desc=f"Val epoch {epoch}")
            all_targets = []
            all_preds = []
            with torch.no_grad():
                for data in val_loop:
                    data = data.to(self.device)
                    target = data.y.to(torch.float32)
                    pred = self.forward(data)
                    all_targets.append(target.detach().cpu().numpy())
                    all_preds.append(pred.detach().cpu().numpy())
                    loss = self.compute_loss(pred, target)
                    val_loss_buffer.append(loss.item())
                    val_loop.set_postfix(loss=np.mean(val_loss_buffer))

            all_preds = np.concatenate(all_preds, axis=0).reshape(-1)
            all_targets = np.concatenate(all_targets, axis=0).reshape(-1)
            print(self.compute_auc(all_preds, all_targets))
            val_loop.set_postfix(loss=self.compute_auc(all_preds, all_targets))

In [18]:
model = MoleculeCrusher()
model.to("cuda")
model.train(epochs=500, train_dataloader=train_dataloaders[0], val_dataloader=val_dataloaders[0])

Train epoch 0:   0%|          | 0/138 [00:00<?, ?it/s]

Val epoch 0: 100%|██████████| 35/35 [00:00<00:00, 183.97it/s, loss=0.66] 


0.6998775861163622


Val epoch 1: 100%|██████████| 35/35 [00:00<00:00, 182.41it/s, loss=0.632]


0.7090586273892039


...

0.7225881511237136


Val epoch 19: 100%|██████████| 35/35 [00:00<00:00, 174.74it/s, loss=0.603]


0.7496980676328502


Train epoch 20:  61%|██████    | 84/138 [00:00<00:00, 97.02it/s, loss=0.648]


KeyboardInterrupt: 

In [ ]:
model.train(epochs=4000, train_dataloader=train_dataloaders[0], val_dataloader=val_dataloaders[0])

In [123]:
val_sample = next(iter(val_dataloaders[0])).to(model.device)
pred = model.predict(val_sample)
pred = (pred >= 0.5).to(torch.float32)
print(pred.to(torch.long))
print(val_sample.y)

tensor([0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0,
        0, 0, 1, 0, 0, 1, 1, 0], device='cuda:0')
tensor([1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1,
        0, 0, 1, 0, 0, 1, 1, 0], device='cuda:0')
